In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import requests
import pymongo

In [2]:
 executable_path = {'executable_path': 'chromedriver.exe'}
 browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [ ]:
dir(browser)

# NASA Mars News

In [ ]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
     
news_title = soup.find_all('div', class_='content_title')
news_p = soup.find_all('div', class_='rollover_description_inner')


In [ ]:
for title in news_title: 
    print('-------------')
    print(title.text)

In [ ]:
for description in news_p: 
    print('-------------')
    print(description.text)

In [ ]:
print(len(news_p))

In [ ]:
print(len(news_title))

# JPL Mars Space Images - Featured Image

In [3]:
featured_image_url= 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(featured_image_url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [4]:
featured_image = soup.find ('article')['style'].replace('background-image: url(','').replace(');', '')

In [5]:
#Slice the url to only include the text inside the hyphens 
featured_image = featured_image[1:-1]

In [6]:
parent_url = 'https://www.jpl.nasa.gov'

In [7]:
image_url= parent_url + featured_image 

In [8]:
print(image_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA18358-1920x1200.jpg


# Mars Weather

In [ ]:
mars_weather_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(mars_weather_url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
current_tweet = soup.find_all('div', class_='js-tweet-text-container')

In [ ]:
for tweet in current_tweet: 
    mars_weather_tweet= tweet.find('p').text
    if 'sol' and 'pressure' in mars_weather_tweet: 
        print(mars_weather_tweet)
        print('-------------')

# Mars Facts

In [ ]:
mars_facts_url= 'https://space-facts.com/mars/'
browser.visit(mars_facts_url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
mars_facts_second_table = pd.read_html(mars_facts_url)
mars_facts_second_table[1]

# Mars Hemispheres

In [3]:
mars_hemispheres_url= 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(mars_hemispheres_url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [4]:
#gather all of the links 
parent_url = 'https://astrogeology.usgs.gov/'
hemisphere_image_title = []
hem_name = soup.find_all('h3')

for link in hem_name:
    hemisphere_image_title.append(link.text)

hemisphere_image_title





['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [5]:
hemisphere_image_url = []

for hem in hemisphere_image_title: 
    hem_dict = {
        'title': [],
        'img_url': []
    }
    
    #find your image
    browser.click_link_by_partial_text(hem)
    
    url = browser.find_by_text('Sample')['href']
    
    hem_dict['img_url']= url
    
    hem_dict['title'] = hem 
    
    hemisphere_image_url.append(hem_dict)
    
    browser.visit(mars_hemispheres_url)
    
    

In [6]:
hemisphere_image_url

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

# Establish MongoDB Connection

In [25]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [ ]:
# Define database and collection
db = client.mars_news_db
collection = db.newsheadlines

In [ ]:
db.newsheadlines.drop()

In [ ]:
item_list = []
for item in zip(news_title, news_p):
    item_list.append({
        'news_title': item[0].text, 
        'news_p': item[1].text
    })

In [ ]:
# Insert dictionary into MongoDB as a document
collection.insert(item_list)

In [ ]:
print(item_list)

In [26]:
#DB FOR IMAGES
db = client.mars_news_db
collection = db.image_url


In [45]:
db.image_url.drop()

In [46]:
imageurl_list = []
imageurl_list.append({
        'image_url': image_url
    })

In [47]:
# Insert dictionary into MongoDB as a document
collection.insert(imageurl_list)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


[ObjectId('5d75db3a4926c8fccc8b5744')]